# Example 1: A Quick First Estimate

This notebook covers the most basic use case of the `NeuralMI` library: getting a single, quick estimate of mutual information between two variables.

**Goal:**
1.  Introduce the main `neural_mi.run` function.
2.  Use a simple dataset where the ground truth MI is known analytically.
3.  Compare our estimate to the ground truth to verify the library is working.

## 1. Imports

We'll need `torch` for data handling, our `run` function, and the data generator.

In [ ]:
import torch
import numpy as np
import neural_mi as nmi

## 2. Generating the Data

We will use the `generate_correlated_gaussians` function from the `datasets` module. This function creates two multidimensional Gaussian variables, `X` and `Y`, where we can precisely specify the mutual information between them in **bits**.

The analytical formula for MI between two multivariate Gaussians is:

$$ I(X;Y) = -\frac{1}{2} \log_2 \det(\Sigma_{XY}) $$

Where $\Sigma_{XY}$ is the correlation matrix. Our generator function handles this for us. Let's create data with a known MI of **2.0 bits**.

In [ ]:
# --- Dataset Parameters ---
n_samples = 5000
dim = 5
ground_truth_mi_bits = 2.0

# --- Generate Data ---
x_data, y_data = nmi.datasets.generate_correlated_gaussians(
    n_samples=n_samples, 
    dim=dim, 
    mi=ground_truth_mi_bits
)

# Reshape for the library: [n_samples, n_channels, n_features]
# In this simple case, we have 1 channel and 'dim' features.
x_data = x_data.reshape(n_samples, 1, dim)
y_data = y_data.reshape(n_samples, 1, dim)

print(f"Generated X data shape: {x_data.shape}")
print(f"Generated Y data shape: {y_data.shape}")

## 3. Defining the Analysis Parameters

The `run` function requires a `base_params` dictionary. This tells the internal `Trainer` how to configure the neural network and the training process. For a quick estimate, we don't need to be too fussy, but we still need to provide the essentials.

In [ ]:
base_params = {
    'n_epochs': 50,          # Number of training epochs
    'learning_rate': 1e-3,   # Learning rate for the optimizer
    'batch_size': 128,       # Batch size for training
    'patience': 5,           # Early stopping patience
    
    # --- Network Architecture ---
    'embedding_dim': 16,     # Dimensionality of the learned embeddings
    'hidden_dim': 64,        # Number of units in hidden layers
    'n_layers': 2            # Number of hidden layers in the MLP
}

## 4. Running the MI Estimation

Now we call the main `nmi.run` function. We specify `mode='estimate'` for a single run with the fixed parameters we defined above.

By default, the function returns the MI value in **bits**, which is convenient for comparison with our ground truth. You can request 'nats' by setting `output_units='nats'`.

In [ ]:
estimated_mi_bits = nmi.run(
    x_data=x_data,
    y_data=y_data,
    mode='estimate',
    base_params=base_params,
    output_units='bits' # This is the default, but we're explicit here.
)

print(f"\n--- Results ---")
print(f"Ground Truth MI:      {ground_truth_mi_bits:.3f} bits")
print(f"Estimated MI:         {estimated_mi_bits:.3f} bits")
print(f"Estimation Error:     {abs(estimated_mi_bits - ground_truth_mi_bits):.3f} bits")

## 5. Conclusion

Success! The estimated MI is very close to the ground truth value we specified. This confirms that the core estimation engine is working correctly.

In the next example, we'll tackle a more complex problem where the relationship between X and Y isn't instantaneous, introducing the need to process our data before estimation.